In [1]:
# 11/22 Get Raw Player Bios for Harvard
# Cameron Yick

# For data manipulation
import os
import pandas as pd
import matplotlib.pyplot as plt
# import seaborn as sns
PROJ_ROOT = os.pardir

# For Scraping
from bs4 import BeautifulSoup
import requests
import time

# For munging
import re
import json

# Backoff time for large scrapes
THROTTLE_TIME = .05

# ipython magics
%load_ext watermark
%matplotlib inline
# a nice bonus for making progress bars in ipython

from tqdm import tqdm, tqdm_pandas, tqdm_notebook


In [2]:
%watermark -a "Cameron Yick" -d -t -v -p pandas,seaborn,matplotlib,bs4 -g

Cameron Yick 2016-11-22 20:41:18 

CPython 2.7.12
IPython 5.1.0

pandas 0.18.1
seaborn 0.7.1
matplotlib 1.5.3
bs4 4.5.1
Git hash: 5843cde364ea00571b167c2c7dcfb5669ce09f28


In [3]:
# Initialize reused objects
tqdm.pandas()
s = requests.Session()

In [4]:
playerPath = os.path.join(PROJ_ROOT, "data", "interim", "harvard_players.json")

In [5]:
df = pd.read_json(playerPath)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10896 entries, 0 to 9999
Data columns (total 17 columns):
B/T            488 non-null object
City           10896 non-null object
Event          1418 non-null object
High School    10896 non-null object
Hometown       10896 non-null object
Ht.            5445 non-null object
Name           10896 non-null object
Name_link      10896 non-null object
No.            6265 non-null object
Position       6490 non-null object
Region         10896 non-null object
S/C            580 non-null object
Weapon         243 non-null object
Wt.            4251 non-null object
Yr.            10896 non-null object
season         10896 non-null object
sport          10896 non-null object
dtypes: object(17)
memory usage: 1.5+ MB


In [15]:
# There is are 8618 unique bio links... hm. Scrape now, clean later.
df['Name_link'].unique().shape

(8618L,)

In [16]:
df['Name_link'].head()

0                /sports/m-basebl/mtt/brenner_ryan00.html
1                /sports/m-basebl/mtt/castaldi_joe00.html
10                 /sports/m-basebl/mtt/ludwig_pat00.html
100       /sports/m-basebl/2016-17/bios/cecere_bobby_a3p9
1000    /sports/m-crewlt/2015-16/bios/sanderson_eric_edr3
Name: Name_link, dtype: object

In [18]:
# Let's make some data soup!

def makeAthleteSoup(link, session=s):
#     print("."),
    if not link:
        return None
    time.sleep(THROTTLE_TIME)
    result = session.get("http://www.yalebulldogs.com" + link)
    if result.status_code is 200:
        return BeautifulSoup(result.content, 'lxml').find('div', class_="bio-wrap")
    else:
        print("Site is down!")

In [20]:
bios = df['Name_link'].unique()

In [22]:
bios = pd.Series(bios)

In [ ]:
bios.to_csv()

In [23]:
soups = bios.progress_apply(lambda x: makeAthleteSoup(x))

  7%|█████▎                                                                       | 596/8618 [07:38<2:06:13,  1.06it/s]

KeyboardInterrupt: 